Importing and preprocessing the data

In [1]:

!pip uninstall matplotlib
!pip install matplotlib==3.0.2
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import re
from sklearn import tree
from sklearn.model_selection import train_test_split
#from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train

  Using cached matplotlib-3.0.2.tar.gz (36.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for matplotlib
Failed to build matplotlib


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [533 lines of output]
  Edit setup.cfg to change the build options
  
  BUILDING MATPLOTLIB
              matplotlib: yes [3.0.2]
                  python: yes [3.11.5 | packaged by Anaconda, Inc. | (main,
                          Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]]
                platform: yes [win32]
  
  REQUIRED DEPENDENCIES AND EXTENSIONS
                   numpy: yes [version 1.24.3]
        install_requires: yes [handled by setuptools]
                  libagg: yes [pkg-config information for 'libagg' could not
                          be found. Using local copy.]
                freetype: yes [Using unknown version found on system.]
                     png: yes [Using unknown version found on system.]
                   qhull: yes [pkg-config information for 'libqhull' could not
                          be found. Using local copy.]
  
  OPTIO

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [2]:
def extract_titles(name):
    tsearch = re.search('([A-Za-z]+)\.',name)
    if tsearch:
        return tsearch.group(1)
    return ""

def process_data(data):
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
    data['Has_Cabin'] = data["Cabin"].apply(lambda x: 0 if pd.isna(x) else 1)
    data['Fare'] = data['Fare'].fillna(train['Fare'].median())
    data.loc[data['FamilySize'] == 1,'IsAlone'] = 1
    data['IsAlone'] = 0
    data['Embarked'] = data['Embarked'].fillna('S')
    standard = data['Age'].std()
    average = data['Age'].mean()
    nullage = data['Age'].isnull().sum()
    nullagelist = np.random.randint(average-standard,average+standard,size=nullage)
    data.loc[np.isnan(data['Age']),'Age'] = nullagelist
    data['Age'] = data['Age'].astype(int)
    data['Title'] = data['Name'].apply(extract_titles)
    data['Title'] = data['Title'].replace('Mme','Mrs')
    data['Title'] = data['Title'].replace('Ms','Miss')
    data['Title'] = data['Title'].replace('Mlle','Miss')
    data['Sex'] = data['Sex'].map({'female': 0,'male': 1}).astype(int)
    tmap = {"Mr": 1,"Master": 2,"Mrs": 3,"Miss": 4}
    data['Title'] = data['Title'].map(tmap)
    data['Title'] = data['Title'].fillna(0)
    data['Embarked'] = data['Embarked'].map({'S': 0,'C': 1,'Q': 2}).astype(int)
    data['Fare'] = pd.cut(data['Fare'], bins=[0,10,20,30,np.inf],labels=[0,1,2,3])
    data['Fare'] = data['Fare'].cat.codes
    data['Age'] = pd.cut(data['Age'], bins=[0,16,32,48,64,np.inf],labels=[0,1,2,3,4])
    data['Age'] = data['Age'].cat.codes
    drop_elements = ['PassengerId','Name','Ticket','Cabin','SibSp','Sex']
    data.drop(drop_elements, axis=1, inplace=True)

process_data(train)
process_data(test)

train


,Survived,Pclass,Age,Parch,Fare,Embarked,FamilySize,Has_Cabin,IsAlone,Title
0,0,3,1,0,0,0,2,0,0,1.0
1,1,1,2,0,3,1,2,1,0,3.0
2,1,3,1,0,0,0,1,0,0,4.0
3,1,1,2,0,3,0,2,1,0,3.0
4,0,3,2,0,0,0,1,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,0,1,0,1,0,0,0.0
887,1,1,1,0,2,0,1,1,0,4.0
888,0,3,2,2,2,0,4,0,0,4.0
889,1,1,1,0,2,1,1,1,0,1.0


Decision Trees

In [3]:
class DecisionTree:
    def __init__(self,impurity='gini',max_depth=10,min_samples_split=4,min_samples_leaf=2):
        self.impurity = impurity
        self.max_depth= max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criterion_functions = {'gini': lambda y: 1 - np.sum((np.unique(y,return_counts=True)[1]/len(y))**2),
                                'entropy': lambda y: -np.sum((np.unique(y, return_counts=True)[1]/len(y)) * np.log2(np.unique(y,return_counts=True)[1]/len(y))),
                                'misclassification_rate': lambda y: 1 - np.max(np.unique(y,return_counts=True)[1]/len(y))}
        self.homogeneity_measure = self.criterion_functions.get(impurity, self.criterion_functions['gini'])

    def fit(self,X,y):
        if isinstance(X,pd.DataFrame):
            X = X.values
        if isinstance(y,pd.Series):
            y = y.values
        self.tree = self.split(X,y,0)

    def split(self,X,y,d):
        n_samples,nfeatures = X.shape
        if n_samples<self.min_samples_split or d>self.max_depth:
            return {'leaf_value':np.round(np.mean(y))}
        blx,bly,brx,bry = None,None,None,None
        bf,bt,bi = None,None,np.inf
        for bestfeatures in range(nfeatures):
            feat = np.unique(X[:,bestfeatures])
            for i in feat:
                rindex = X[:,bestfeatures]>i
                lindex = X[:,bestfeatures]<=i
                if len(y[rindex])>0 and len(y[lindex])>0:
                    ly,ry = y[lindex],y[rindex]
                    nl,nr = len(ly),len(ry)
                    all = nl+nr
                    bad = (nl/all)*self.homogeneity_measure(ly) + \
                               (nr/all)*self.homogeneity_measure(ry)
                    if bad<bi:
                        bf,bt,bi = bestfeatures,i,bad
                        blx,bly = X[lindex],y[lindex]
                        brx,bry = X[rindex],y[rindex]
        if bf is not None:
            obj = {'feature':bf,'threshold':bt,
                    'left': self.split(blx,bly,d+1),
                    'right':self.split(brx,bry,d+1)}
            return obj
        return {'leaf_value':np.round(np.mean(y))}

    def predicted(self,x,obj):
        while 'leaf_value' not in obj:
            if x[obj['feature']] <= obj['threshold']:
                obj = obj['left']
            else:
                obj = obj['right']
        return obj['leaf_value']

    def predict(self,X):
        X = pd.DataFrame(X)
        X.reset_index(drop=True,inplace=True)
        ypred = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            ypred[i] = self.predicted(X.loc[i],self.tree)
        return ypred


In [5]:
y = train['Survived']
X = train.drop('Survived',axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)
decisiontree = DecisionTree(max_depth=5)
decisiontree.fit(X_train,y_train)
ypred = decisiontree.predict(X_test)
dacc = accuracy_score(y_test,ypred)
print("Accuracy of Decision Tree Model : {:.2f}%".format(dacc*100))
print(y)
from sklearn.metrics import classification_report
print("Classification Report: \n", classification_report(y_test,ypred))

from sklearn.metrics import roc_auc_score

# After fitting and predicting

roc_auc = roc_auc_score(y_test, ypred)
print("ROCAUCdt: ", roc_auc)

Accuracy of Decision Tree Model : 78.73%
0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64
Classification Report: 
               precision    recall  f1-score   support

           0       0.80      0.84      0.82       157
           1       0.76      0.71      0.73       111

    accuracy                           0.79       268
   macro avg       0.78      0.78      0.78       268
weighted avg       0.79      0.79      0.79       268

ROCAUCdt:  0.7762380214609513


Random Forests

In [6]:
class RandomForest:
    def __init__(self,classifier,num_trees,min_features):
        self.classifier = classifier
        self.num_trees = num_trees
        self.min_features = min_features
        self.trees = []
        
    def fit(self,X,y):
        X = X.reset_index(drop=True)
        y = y.reset_index(drop=True)
        for i in range(self.num_trees):
            cnt = np.random.randint(1,X.shape[0]+1)
            sid = np.random.choice(X.index,cnt,replace=True)
            X_b,y_b = X.loc[sid],y.loc[sid]
            feats = np.random.randint(self.min_features,X.shape[1]+1)
            fid = np.random.choice(X.columns,feats,replace=False)
            X_b = X_b[fid]
            tree = self.classifier(max_depth=10,min_samples_split=2,min_samples_leaf=2)
            tree.fit(X_b,y_b) 
            self.trees.append((tree,fid))
            
    def predict(self,X):
        aa = []
        for tree,fid in self.trees:
            xx = X[fid]
            aa.append(tree.predict(xx))
        aa = np.array(aa).astype(int)
        return np.apply_along_axis(lambda x:np.bincount(x).argmax(),axis=0,arr=aa)


randomforest = RandomForest(classifier=DecisionTree,num_trees=100,min_features=4)
randomforest.fit(X_train,y_train)
ypredr = randomforest.predict(X_test)
racc = accuracy_score(y_test,ypredr)
print("Accuracy of Random Forest Model : {:.2f}%".format(racc*100))
print(y)
print("Classification Report: \n", classification_report(y_test,ypredr))
roc_auc = roc_auc_score(y_test, ypredr)
print("ROCAUCrf: ", roc_auc)

Accuracy of Random Forest Model : 79.10%
0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64
Classification Report: 
               precision    recall  f1-score   support

           0       0.78      0.90      0.83       157
           1       0.82      0.64      0.72       111

    accuracy                           0.79       268
   macro avg       0.80      0.77      0.78       268
weighted avg       0.79      0.79      0.79       268

ROCAUCrf:  0.768864405807081


Boosting

In [7]:
from sklearn.metrics import classification_report

class AdaBoost:
    def __init__(self,weak_learner,num_learners,learning_rate):
        self.weak_learner = weak_learner
        self.num_learners = num_learners
        self.learning_rate = learning_rate
    
    def fit(self,X,y):
        self.mm = []
        self.nn = []
        cnt = X.shape[0]
        yy = np.ones(cnt)/cnt                
        for _ in range(self.num_learners):
            ml = self.weak_learner()
            ml.fit(X,y)
            ypred = ml.predict(X)
            grad = np.mean(np.abs(ypred-y)/2*yy)/np.mean(yy)
            if grad>0.5:
                break
            dd = self.learning_rate*np.log((1-grad)/grad)
            self.mm.append(ml)
            self.nn.append(dd)            
            yy *= np.exp(-dd*y*ypred)
            yy /= np.sum(yy)
            
    def predict(self,X):
        n = X.shape[0]
        ypred = np.zeros(n)        
        for i in range(len(self.mm)):
            m1 = self.mm[i]
            m2 = self.nn[i]
            ypred += m2*m1.predict(X)       
        return np.sign(ypred)


ab = AdaBoost(weak_learner=DecisionTree,num_learners=200,learning_rate=0.1)
ab.fit(X_train,y_train)
ypredb = ab.predict(X_test)
bacc = accuracy_score(y_test,ypredb)
print("Accuracy of Boosting Model : {:.2f}%".format(bacc*100))
print(y)
print("Classification Report: \n", classification_report(y_test,ypredb))
roc_auc = roc_auc_score(y_test, ypredb)
print("ROCAUCrf: ", roc_auc)

Accuracy of Boosting Model : 77.24%
0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64
Classification Report: 
               precision    recall  f1-score   support

           0       0.78      0.85      0.81       157
           1       0.76      0.66      0.71       111

    accuracy                           0.77       268
   macro avg       0.77      0.76      0.76       268
weighted avg       0.77      0.77      0.77       268

ROCAUCrf:  0.7555804211855167


In [8]:
models = pd.DataFrame({'Model': ['Random Forest', 'Decision Tree', 'adaBoost'],
                       'Score': [ dacc, racc, bacc]})


models.sort_values(by = 'Score', ascending = False, ignore_index = True)


,Model,Score
0,Decision Tree,0.791045
1,Random Forest,0.787313
2,adaBoost,0.772388
